# [Multilevel Modeling](https://en.wikipedia.org/wiki/Multilevel_model)
Idea: Decompose the sum of random numbers into its contributions

for a given set of $x_i$ and $a_{ik}$ with
$$x_i = \Sigma_{j=0}^n \Sigma_{k=1}^{m_j} a_{ik}y_{jk}$$ 
with
* $m_j$ being the number of contributers of layer j
* $y_{jk}$ being the k-th contribution of layer j
* $m_0 = 1$ by default
* $a_{ik} \in \{0, 1\}$
* $\Sigma a_{ik} = 1$ only one contributer per layer

calculate the Distributions $y_{k} \sim N(μ_{k}, σ_{k})$
Boundary conditions:
* $\Sigma _{k} μ_k = 0$ via $μ_{m_j} = -\Sigma_{k = 1}^{m_j-1} μ_{k}$


Goal of this workbook to gather the basics
* which package to use
* how to solve
* how to model

Answerː Use pymc with nutpie as a solver :-)

In [14]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
import polars as pl

from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
import plotly.express as px

%matplotlib inline
plt.style.use("default")


## Initialize the data

## setup the individual random generators

In [2]:
def align_coefficients(coefs):
    """ensure that the average over all contributers per level is 0"""
    mem = {}
    for level, lcoefs in coefs.items():
        avg = np.mean([m for (m, s) in lcoefs.values()])
        mem[level] = {k: (m - avg, s) for k, (m, s) in lcoefs.items()}

    return mem


def gen_contributer_coefficients(n_contributers, contributer_avg, contributer_sig):
    """creates for n_contributers the average and standard deviation
    Input:
    * n_contributers: list with number of influencers for each level
    * contributer_avg: average for the average of the influencers
    * contributer_sig: average for the stdev for the influencers (lognormal distribution)

    Output:
    Dictionary {level : { influencer: (mu, sigma)}} with the coefficients for every
    influencer in each level
    """
    # assert n_contributers[0] == 1, "First level is allowed to have one contributer"
    return align_coefficients(
        {
            lvl: {
                i: (
                    np.random.normal(contributer_avg),
                    np.random.lognormal(contributer_sig),
                )
                for i in range(num)
            }
            for lvl, num in enumerate(n_contributers)
        }
    )


def print_coefficients(contributer_coefficients):
    contributer_coefficients = align_coefficients(contributer_coefficients)
    for m, stage in contributer_coefficients.items():
        for i, (mu, sig) in stage.items():
            print(f"Stage {m}: Contributer {i} mu={mu:.2f}, sig={sig:.2f}")

In [3]:
# easy as a start
cc_02 = {0: {0: (0, 5)}, 1: {0: (0, 1), 1: (0, 1), 2: (0, 1), 3: (4, 2)}}
cc_01 = {0: {0: (0, 5)}, 1: {0: (1, 1), 1: (-1, 1)}}

In [4]:
print_coefficients(cc_02)

Stage 0: Contributer 0 mu=0.00, sig=5.00
Stage 1: Contributer 0 mu=-1.00, sig=1.00
Stage 1: Contributer 1 mu=-1.00, sig=1.00
Stage 1: Contributer 2 mu=-1.00, sig=1.00
Stage 1: Contributer 3 mu=3.00, sig=2.00


## Create the Data

In [5]:
def gen_data(contributer_coefficients, n_samples):
    """generate random data.
    The first level defines the baseline that holds for all random numbers

    Inputs:
    * n_samples: number of samples
    * contributer_coefficients: dictionary with (mean,sig) per level per contributer

    Outputs:
    * data: array with the final number
    * contributers: matrix defining the contributers, first column is for the first level"""

    # data = np.random.normal(gen_avg, gen_sig, n_samples)
    data = np.zeros((n_samples,))
    contributers = np.zeros((n_samples, len(contributer_coefficients)))
    for lvl, cdict in contributer_coefficients.items():
        # print(f"creating level {lvl}")
        lvl_influencers = len(cdict)  # number of influencers in this level
        lvldata = np.zeros((n_samples, lvl_influencers))

        for i, (mu, sig) in cdict.items():
            lvldata[:, i] = np.random.normal(mu, sig, n_samples)

        selection = np.random.randint(low=0, high=lvl_influencers, size=(n_samples))
        contributers[:, lvl] = selection

        data += np.array([lvldata[row, col] for row, col in enumerate(selection)])
        # Note: The first level
    return data, contributers.astype(int)


def run(n_iterations, n_samples, coefficients):
    mem = []

    for run_id in range(n_iterations):
        data, contributers = gen_data(
            n_samples=n_samples, contributer_coefficients=coefficients
        )

        pipe = Pipeline(
            [
                ("ohe", OneHotEncoder(sparse_output=False)),
                ("regression", BayesianRidge(fit_intercept=False)),
            ]
        )
        pipe.fit(contributers, data)

        df = (
            pl.DataFrame(
                {"parameter": pipe[0].get_feature_names_out(), "value": pipe[1].coef_}
            )
            .with_columns(pl.col("parameter").str.split("_").alias("splits"))
            .with_columns(
                pl.col("splits").list.first().alias("level"),
                pl.col("splits").list.last().alias("con"),
                pl.lit(run_id).alias("run_id"),
            )
            .drop("splits")
        )
        mem.append(df)

    return pl.concat(mem)

# Modeling

In [6]:
n_contributers = [2, 3, 5, 10, 10]

coefficients = gen_contributer_coefficients(
    n_contributers=n_contributers, contributer_avg=2, contributer_sig=2
)

In [7]:
data = run(n_iterations=100, n_samples=1000, coefficients=coefficients)

In [8]:
data

parameter,value,level,con,run_id
str,f64,str,str,i32
"""x0_0""",-0.476551,"""x0""","""0""",0
"""x0_1""",-0.338778,"""x0""","""1""",0
"""x1_0""",-0.528082,"""x1""","""0""",0
"""x1_1""",-2.201473,"""x1""","""1""",0
"""x1_2""",1.914226,"""x1""","""2""",0
…,…,…,…,…
"""x4_5""",-0.028744,"""x4""","""5""",99
"""x4_6""",0.007332,"""x4""","""6""",99
"""x4_7""",0.021247,"""x4""","""7""",99


# Dash

In [44]:
app = Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options


fig = px.box(data, x="parameter", y="value", color="level")

app.layout = html.Div(
    children=[
        html.H1(children="Hello Dash"),
        html.Div(
            children="""
        Dash: A web application framework for your data.
    """
        ),
        dcc.Graph(id="example_graph", figure=fig),
        dcc.Checklist(
            id="checklist",
            options=[
                {"label": f"{value}", "value": value}
                for value in data["level"].unique()
            ],
            value=[],
            labelStyle={"display": "inline-block"},
        ),
    ]
)


color_dict = {
    lvl: c
    for i, (lvl, c) in enumerate(
        zip(data["level"].unique(), px.colors.qualitative.Plotly)
    )
}


@app.callback(
    Output("example_graph", "figure"),
    Input("checklist", "value"),
)
def update_graph(selected_levels):
    filtered_df = data.filter(pl.col("level").is_in(selected_levels))

    fig = px.box(
        filtered_df,
        x="parameter",
        y="value",
        color="level",
        hover_name="run_id",
        color_discrete_map=color_dict,
    )

    fig.update_layout(transition_duration=500)

    return fig

In [45]:
app.run(debug=True)